
<body>
<h1>ISDA 111 - Video Game Data Analytics Project</h1>
<p>Hi team, This is the starter document we can use.</p>
<h2>Team Members: add your name here</h2>
<p>Chris Alcala - Handimexi</p>
<p>Wilson Huynh - mybedro0m</p>
<p>Fernando Valle - Jando Fett</p>

</body>



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

In [7]:
df = pd.read_csv("game_data_all.csv")
df.columns

Index(['Unnamed: 0', 'game', 'link', 'release', 'peak_players',
       'positive_reviews', 'negative_reviews', 'total_reviews', 'rating',
       'primary_genre', 'store_genres', 'publisher', 'developer',
       'detected_technologies', 'store_asset_mod_time', 'review_percentage',
       'players_right_now', '24_hour_peak', 'all_time_peak',
       'all_time_peak_date'],
      dtype='object')

In [8]:
df

       Unnamed: 0                              game           link  \
0               0                       Pizza Tower  /app/2231450/   
1               1                   Resident Evil 4  /app/2050650/   
2               2  The Murder of Sonic the Hedgehog  /app/2324650/   
3               3                Pineapple on pizza  /app/2263010/   
4               4                        Hi-Fi RUSH  /app/1817230/   
...           ...                               ...            ...   
67566       67566                   Space Empires V     /app/1690/   
67567       67567  Iron Warriors: T-72 Tank Command     /app/1670/   
67568       67568          Gumboy: Crazy Adventures     /app/2520/   
67569       67569                   Shattered Union     /app/3960/   
67570       67570               Poker Superstars II     /app/4100/   

          release  peak_players  ...  review_percentage  players_right_now  \
0      2023-01-26          4529  ...               98.0              3,101   
1  